In [ ]:
import pandas as pd
import mysql.connector
import re
conn = mysql.connector.connect(username= "root",
                               host = "127.0.0.1",
                               password = "Admin@123",
                               database = "dataspark")

cursor = conn.cursor()

In [ ]:
df_sales = pd.read_csv('Sales.csv')

In [ ]:
df_sales.columns = df_sales.columns.str.lower()
df_sales.columns = df_sales.columns.str.replace(" ","_")
df_sales.columns
df_sales['order_date']= pd.to_datetime(df_sales['order_date'])
df_sales['delivery_date']= pd.to_datetime(df_sales['delivery_date'])

In [ ]:
df_sales["shipping_days"] = (df_sales['delivery_date'] - df_sales['order_date']).dt.days

In [ ]:
df_sales['delivery_date'] = df_sales['delivery_date'].fillna((df_sales['order_date'] + pd.Timedelta(days=4)))

In [ ]:
df_sales["shipping_days"] = (df_sales['delivery_date'] - df_sales['order_date']).dt.days

In [ ]:
df_sales.head()

In [ ]:
insert_sales_values = '''
    INSERT INTO sales (
        order_number, 
        line_item, 
        order_date, 
        delivery_date, 
        customerkey, 
        storekey, 
        productkey, 
        quantity, 
        currency_code, 
        shipping_days
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
'''

# Insert data into the sales table
for index, row in df_sales.iterrows():
    cursor.execute(insert_sales_values, (
        row['order_number'],
        row['line_item'],
        row['order_date'],
        row['delivery_date'],
        row['customerkey'],
        row['storekey'],
        row['productkey'],
        row['quantity'],
        row['currency_code'],
        row['shipping_days']
    ))

conn.commit()    